In [12]:
from __future__ import print_function
import pickle 
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from sub import subMNIST

In [13]:
trainset_labeled = pickle.load(open("train_labeled.p", "rb")) 
train_loader  = torch.utils.data.DataLoader(trainset_labeled, batch_size=64, shuffle=True, num_workers=2)

validset = pickle.load(open("validation.p", "rb"))
valid_loader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=True, num_workers=2)

In [14]:
#utility functions
def rand(x, level = 1e-3):
    return x + torch.randn(x.size()) * level

def norm_weights_2d(size):
    return BatchNorm2d(nn.Linear(size))
def norm_weights_1d(size):
    return BatchNorm2d(nn.Linear(size))

def reluN(x, level = 1e-3):
    y = x + torch.randn(x.size()) * level

In [19]:
def norm(size, dim = 1):
    if dim == 1:
        return torch.nn.BatchNorm1d(size)
    else:
        return torch.nn.BatchNorm2d(size)
def linear(indim, outdim):
    return nn.Linear(indim, outdim)
def var(tens):
    return torch.autograd.Variable(tens)
def randn(size):
    return torch.randn(size)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = [1000, 600, 300, 10]
        layers = self.layers
        self.encs = [28*28, layers[0]]
        self.lats = []
        self.encnorms = []
        self.decs = []
        self.decnorms = []
        for idx, l in enumerate(layers[:-1]):
            self.lats.append(linear(l, l))
            self.encnorms.append(norm(l))
            self.encs.append(linear(l, layers[idx+1]))
            self.decs.append(linear(layers[idx+1], l))
            self.decnorms.append(norm(l))
        self.batch_size = 64
        self.weights = [self.encs, self.lats, self.decs]
        

    def forward(self, x, v = 0, noise=1e-3):
        self.eps = noise
        self.batch_size = x.size()[0]
        bs = self.batch_size
        #enc= F.relu(self.enc4(F.relu(self.enc3(F.relu(self.enc2(F.relu(self.enc1(x))))))))
        corrupted = []
        corruptedout = []
        for idx, l in enumerate(self.layers):
            if idx == 0:
                corrupted.append(self.encnorms[l](self.encs[l](x)))
            else:
                corrupted.append(self.encnorms[l](self.encs[l](corruptedout[-1])))
            corruptedout.append(F.relu(self.encs[l] + self.encnorms[l].weight.unsqueeze(0).expand(
                    bs, l) * var(randn(corrupted[-1].size()) * self.eps) +
                    self.encnorms[0].bias.unsqueeze(0).expand(bs, l)))
            
        encout = F.softmax(corruptedout[-1])
        
        clean = [x]
        for norm, enc in zip(self.encnorms, self.encs):
            clean.append(F.relu(norm(enc(clean[-1]))))
        
        decout = [encout]
        decin = []
        for idx in range(len(self.layers) - 1, -1, -1):
            dec = self.decs[idx]
            lat = self.lats[idx]
            decnorm = self.decnorms[idx+1]
            decin.append(dec(decout[-1]) + lat(corruptedout[idx]))
            decout.append(F.relu(decin[-1] + decnorm.weight.unsqueeze(0).expand(bs, self.layers[idx]) *
                    var(randn(decin[-1].size()) * self.eps) + \
                    decnorm.bias.unsqueeze(0).expend(bs, self.layers[idx])))
        weight_reg = 0
        for w_list in self.weights:
            for w in w_list:
                weight_reg += (w.weight**2).mean()/100
        
        yhat = F.log_softmax(corruptedout[-1])
        
        encode_err = 0
        enc_weight = 5
        enc_decay = .2
        for c, d in zip(clean[-2::-1], decin):
            encode_err += enc_weight * ((c - d)**2).mean()
        return yhat, weight_reg + encode_err
        

model = Net()
params = list(model.parameters())

if 1:
    print(model)
    print('Models has {} learnable paramater:'.format(len(params)))
    [print('parameter {} has a size of {}'.format(i+1, params[i].size())) for i in range(len(params))]

Net (
)
Models has 0 learnable paramater:


In [16]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

IndexError: list index out of range

In [59]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target) # Wrap them in Variable 
        optimizer.zero_grad() # Zero the parameter gradients
        data = data.view(data.size()[0], 28*28)
        outputs = model(data,noise=1e-2) # Forward 
        output = outputs[0]
        US = outputs[1]
        loss = F.nll_loss(output, target) + US*.1
        loss.backward() 
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [60]:
def test(epoch, valid_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:

        data, target = Variable(data, volatile=True), Variable(target)
        data = data.view(data.size()[0], 28*28)
        outputs = model(data, noise = 0)
        output = outputs[0]
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [61]:
for epoch in range(1, 30):
    train(epoch)
    test(epoch, valid_loader)

Train Epoch: 1 [0/3000 (0%)]	Loss: 2.397224
Train Epoch: 1 [640/3000 (21%)]	Loss: 1.519292
Train Epoch: 1 [1280/3000 (43%)]	Loss: 1.349676
Train Epoch: 1 [1920/3000 (64%)]	Loss: 1.206272
Train Epoch: 1 [2560/3000 (85%)]	Loss: 1.036737

Test set: Average loss: 0.7336, Accuracy: 8942/10000 (89%)

Train Epoch: 2 [0/3000 (0%)]	Loss: 0.859665
Train Epoch: 2 [640/3000 (21%)]	Loss: 0.651262
Train Epoch: 2 [1280/3000 (43%)]	Loss: 0.693339
Train Epoch: 2 [1920/3000 (64%)]	Loss: 0.505255
Train Epoch: 2 [2560/3000 (85%)]	Loss: 0.446675

Test set: Average loss: 0.4306, Accuracy: 9278/10000 (93%)

Train Epoch: 3 [0/3000 (0%)]	Loss: 0.394293
Train Epoch: 3 [640/3000 (21%)]	Loss: 0.412965
Train Epoch: 3 [1280/3000 (43%)]	Loss: 0.304297
Train Epoch: 3 [1920/3000 (64%)]	Loss: 0.339598
Train Epoch: 3 [2560/3000 (85%)]	Loss: 0.257155

Test set: Average loss: 0.3471, Accuracy: 9367/10000 (94%)

Train Epoch: 4 [0/3000 (0%)]	Loss: 0.294940
Train Epoch: 4 [640/3000 (21%)]	Loss: 0.221244
Train Epoch: 4 [1280/